In [22]:
%pip install pyod

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu


Looking in indexes: https://download.pytorch.org/whl/cpu
  Using cached https://download.pytorch.org/whl/cpu/torchvision-0.24.0%2Bcpu-cp313-cp313-win_amd64.whl.metadata (6.1 kB)
  Using cached https://download.pytorch.org/whl/cpu/torchaudio-2.9.0%2Bcpu-cp313-cp313-win_amd64.whl.metadata (7.0 kB)
Note: you may need to restart the kernel to use updated packages.


ERROR: HTTP error 403 while getting https://download.pytorch.org/whl/cpu/torchvision-0.24.0%2Bcpu-cp313-cp313-win_amd64.whl#sha256=24ae40dd443486bd32118a431ee4f38a23455665c477bb42136415e2a1c940e5 (from https://download.pytorch.org/whl/cpu/torchvision/)

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: 403 Client Error: Forbidden for url: https://download.pytorch.org/whl/cpu/torchvision-0.24.0%2Bcpu-cp313-cp313-win_amd64.whl


In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

from pyod.models.so_gaal import SO_GAAL

In [25]:
data = pd.read_csv('Datasets/demo_test_weekly_data_2810.csv')
data.head()


,provider_id,provider_office_city_nm,provider_office_state_cd,provider_office_street_address,provider_office_zip_cd,provider_office_zip_cd_3_digit,provider_urf_designation,provider_hsr,medicare_provider_id_provider,national_provider_id_provider,...,city,latitude,longitude,state,zip_cd,zip_cd_3_digit,urf_designation,hsr,tax_id_type,anomalies_present
0,45795.0,Williamton,AS,4836 Elizabeth Trail,87020.0,800.0,Rural,HSR46,998873283.0,1.043041e+09,...,Bryantstad,-54.348750,9.106338,LA,5301.0,873.0,Rural,76.0,SSN,False
1,47316.0,East Kathleenstad,SD,179 Yates Causeway,92664.0,711.0,Rural,HSR30,361233369.0,1.245062e+09,...,Lake Robert,79.337282,-81.859771,KS,74391.0,208.0,Rural,80.0,EIN,False
2,12404.0,Lake Jacobside,VT,77038 Miller Cove,5885.0,54.0,Rural,HSR11,647634854.0,1.326523e+09,...,Lake Robertville,72.813079,-146.103737,VT,17017.0,145.0,Rural,2.0,EIN,False
3,22381.0,New Margaret,PA,17804 Steven Square,68430.0,598.0,Frontier,HSR48,941060204.0,1.790516e+09,...,Port Daniel,-77.470106,-44.396921,TN,74551.0,21.0,Frontier,53.0,EIN,False
4,50223.0,Port Dylan,MS,10583 Atkinson Radial,82887.0,242.0,Frontier,HSR95,272260114.0,1.265616e+09,...,Nancyshire,-24.643452,72.918564,NH,26842.0,70.0,Frontier,2.0,SSN,False


In [30]:
df.shape

(10010, 77)

In [26]:
df= data.copy()
df.head()

,provider_id,provider_office_city_nm,provider_office_state_cd,provider_office_street_address,provider_office_zip_cd,provider_office_zip_cd_3_digit,provider_urf_designation,provider_hsr,medicare_provider_id_provider,national_provider_id_provider,...,city,latitude,longitude,state,zip_cd,zip_cd_3_digit,urf_designation,hsr,tax_id_type,anomalies_present
0,45795.0,Williamton,AS,4836 Elizabeth Trail,87020.0,800.0,Rural,HSR46,998873283.0,1.043041e+09,...,Bryantstad,-54.348750,9.106338,LA,5301.0,873.0,Rural,76.0,SSN,False
1,47316.0,East Kathleenstad,SD,179 Yates Causeway,92664.0,711.0,Rural,HSR30,361233369.0,1.245062e+09,...,Lake Robert,79.337282,-81.859771,KS,74391.0,208.0,Rural,80.0,EIN,False
2,12404.0,Lake Jacobside,VT,77038 Miller Cove,5885.0,54.0,Rural,HSR11,647634854.0,1.326523e+09,...,Lake Robertville,72.813079,-146.103737,VT,17017.0,145.0,Rural,2.0,EIN,False
3,22381.0,New Margaret,PA,17804 Steven Square,68430.0,598.0,Frontier,HSR48,941060204.0,1.790516e+09,...,Port Daniel,-77.470106,-44.396921,TN,74551.0,21.0,Frontier,53.0,EIN,False
4,50223.0,Port Dylan,MS,10583 Atkinson Radial,82887.0,242.0,Frontier,HSR95,272260114.0,1.265616e+09,...,Nancyshire,-24.643452,72.918564,NH,26842.0,70.0,Frontier,2.0,SSN,False


Helping Functions

In [27]:
def check_indexes(target_indexes, indexes_to_find):
    idx_list = []
    for idx in indexes_to_find:
        if idx in target_indexes:
            print(f"-- Index {idx} found.")
            idx_list.append(idx)
    return idx_list
        # else:
            # print(f"Index {idx} not found.")

In [28]:

import matplotlib.pyplot as plt

# from .data import check_consistent_shape
# from .data import get_outliers_inliers
from sklearn.utils import check_X_y
from sklearn.utils import check_consistent_length
from sklearn.utils import column_or_1d

def check_consistent_shape(X_train, y_train, X_test, y_test, y_train_pred,
                           y_test_pred):
    """Internal shape to check input data shapes are consistent.

    Parameters
    ----------
    X_train : numpy array of shape (n_samples, n_features)
        The training samples.

    y_train : list or array of shape (n_samples,)
        The ground truth of training samples.

    X_test : numpy array of shape (n_samples, n_features)
        The test samples.

    y_test : list or array of shape (n_samples,)
        The ground truth of test samples.

    y_train_pred : numpy array of shape (n_samples, n_features)
        The predicted binary labels of the training samples.

    y_test_pred : numpy array of shape (n_samples, n_features)
        The predicted binary labels of the test samples.

    Returns
    -------
    X_train : numpy array of shape (n_samples, n_features)
        The training samples.

    y_train : list or array of shape (n_samples,)
        The ground truth of training samples.

    X_test : numpy array of shape (n_samples, n_features)
        The test samples.

    y_test : list or array of shape (n_samples,)
        The ground truth of test samples.

    y_train_pred : numpy array of shape (n_samples, n_features)
        The predicted binary labels of the training samples.

    y_test_pred : numpy array of shape (n_samples, n_features)
        The predicted binary labels of the test samples.
    """

    # check input data shapes are consistent
    X_train, y_train = check_X_y(X_train, y_train)
    X_test, y_test = check_X_y(X_test, y_test)

    y_test_pred = column_or_1d(y_test_pred)
    y_train_pred = column_or_1d(y_train_pred)

    check_consistent_length(y_train, y_train_pred)
    check_consistent_length(y_test, y_test_pred)

    if X_train.shape[1] != X_test.shape[1]:
        raise ValueError("X_train {0} and X_test {1} have different number "
                         "of features.".format(X_train.shape, X_test.shape))

    return X_train, y_train, X_test, y_test, y_train_pred, y_test_pred


def get_outliers_inliers(X, y):
    """Internal method to separate inliers from outliers.

    Parameters
    ----------
    X : numpy array of shape (n_samples, n_features)
        The input samples

    y : list or array of shape (n_samples,)
        The ground truth of input samples.

    Returns
    -------
    X_outliers : numpy array of shape (n_samples, n_features)
        Outliers.

    X_inliers : numpy array of shape (n_samples, n_features)
        Inliers.

    """
    X_outliers = X[np.where(y == 1)]
    X_inliers = X[np.where(y == 0)]
    return X_outliers, X_inliers


def visualize(clf_name, X_train, y_train, X_test, y_test, y_train_pred,
              y_test_pred, show_figure=True,
              save_figure=False):  # pragma: no cover
    """Utility function for visualizing the results in examples.
    Internal use only.

    Parameters
    ----------
    clf_name : str
        The name of the detector.

    X_train : numpy array of shape (n_samples, n_features)
        The training samples.

    y_train : list or array of shape (n_samples,)
        The ground truth of training samples.

    X_test : numpy array of shape (n_samples, n_features)
        The test samples.

    y_test : list or array of shape (n_samples,)
        The ground truth of test samples.

    y_train_pred : numpy array of shape (n_samples, n_features)
        The predicted binary labels of the training samples.

    y_test_pred : numpy array of shape (n_samples, n_features)
        The predicted binary labels of the test samples.

    show_figure : bool, optional (default=True)
        If set to True, show the figure.

    save_figure : bool, optional (default=False)
        If set to True, save the figure to the local.

    """

    def _add_sub_plot(X_inliers, X_outliers, sub_plot_title,
                      inlier_color='blue', outlier_color='orange'):
        """Internal method to add subplot of inliers and outliers.

        Parameters
        ----------
        X_inliers : numpy array of shape (n_samples, n_features)
            Outliers.

        X_outliers : numpy array of shape (n_samples, n_features)
            Inliers.

        sub_plot_title : str
            Subplot title.

        inlier_color : str, optional (default='blue')
            The color of inliers.

        outlier_color : str, optional (default='orange')
            The color of outliers.

        """
        plt.axis("equal")
        plt.scatter(X_inliers[:, 0], X_inliers[:, 1], label='inliers',
                    color=inlier_color, s=40)
        plt.scatter(X_outliers[:, 0], X_outliers[:, 1],
                    label='outliers', color=outlier_color, s=50, marker='^')
        plt.title(sub_plot_title, fontsize=15)
        plt.xticks([])
        plt.yticks([])
        plt.legend(loc=3, prop={'size': 10})

    # check input data shapes are consistent
    X_train, y_train, X_test, y_test, y_train_pred, y_test_pred = \
        check_consistent_shape(X_train, y_train, X_test, y_test, y_train_pred,
                               y_test_pred)

    if X_train.shape[1] != 2:
        raise ValueError("Input data has to be 2-d for visualization. The "
                         "input data has {shape}.".format(shape=X_train.shape))

    X_train_outliers, X_train_inliers = get_outliers_inliers(X_train, y_train)
    X_train_outliers_pred, X_train_inliers_pred = get_outliers_inliers(
        X_train, y_train_pred)

    X_test_outliers, X_test_inliers = get_outliers_inliers(X_test, y_test)
    X_test_outliers_pred, X_test_inliers_pred = get_outliers_inliers(
        X_test, y_test_pred)

    # plot ground truth vs. predicted results
    fig = plt.figure(figsize=(12, 10))
    plt.suptitle("Demo of {clf_name} Detector".format(clf_name=clf_name),
                 fontsize=15)

    fig.add_subplot(221)
    _add_sub_plot(X_train_inliers, X_train_outliers, 'Train Set Ground Truth',
                  inlier_color='blue', outlier_color='orange')

    fig.add_subplot(222)
    _add_sub_plot(X_train_inliers_pred, X_train_outliers_pred,
                  'Train Set Prediction', inlier_color='blue',
                  outlier_color='orange')

    fig.add_subplot(223)
    _add_sub_plot(X_test_inliers, X_test_outliers, 'Test Set Ground Truth',
                  inlier_color='green', outlier_color='red')

    fig.add_subplot(224)
    _add_sub_plot(X_test_inliers_pred, X_test_outliers_pred,
                  'Test Set Prediction', inlier_color='green',
                  outlier_color='red')

    if save_figure:
        plt.savefig('{clf_name}.png'.format(clf_name=clf_name), dpi=300)

    if show_figure:
        plt.show()


def data_visualize(X_train, y_train, show_figure=True,
                   save_figure=False):  # pragma: no cover
    """Utility function for visualizing the synthetic samples generated by
    generate_data_cluster function.

    Parameters
    ----------
    X_train : numpy array of shape (n_samples, n_features)
        The training samples.

    y_train : list or array of shape (n_samples,)
        The ground truth of training samples.

    show_figure : bool, optional (default=True)
        If set to True, show the figure.

    save_figure : bool, optional (default=False)
        If set to True, save the figure to the local.

    """

    def _plot(X_inliers, X_outliers, inlier_color='blue',
              outlier_color='orange'):
        """Internal method to add subplot of inliers and outliers.

        Parameters
        ----------
        X_inliers : numpy array of shape (n_samples, n_features)
            Outliers.

        X_outliers : numpy array of shape (n_samples, n_features)
            Inliers.

        sub_plot_title : str
            Subplot title.

        inlier_color : str, optional (default='blue')
            The color of inliers.

        outlier_color : str, optional (default='orange')
            The color of outliers.

        """
        plt.axis("equal")
        plt.scatter(X_inliers[:, 0], X_inliers[:, 1], label='inliers',
                    color=inlier_color, s=40)
        plt.scatter(X_outliers[:, 0], X_outliers[:, 1],
                    label='outliers', color=outlier_color, s=50, marker='^')
        plt.xticks([])
        plt.yticks([])
        plt.legend(loc='best', prop={'size': 10})

    assert len(X_train) <= 5
    in_colors = ['blue', 'green', 'purple', 'brown', 'black']
    out_colors = ['red', 'orange', 'grey', 'violet', 'pink']
    plt.figure(figsize=(13, 10))
    plt.suptitle("Demo of Generating Data in Clusters", fontsize=15)
    for i, cluster in enumerate(X_train):
        X_train_outliers, X_train_inliers = get_outliers_inliers(cluster,
                                                                 y_train[i])
        _plot(X_train_inliers, X_train_outliers,
              inlier_color=in_colors[i],
              outlier_color=out_colors[i])

    if save_figure:
        plt.savefig()

    if show_figure:
        plt.show()

In [29]:

from sklearn.preprocessing import OneHotEncoder

cols_set = ['provider_id','provider_first_nm_provider', 'provider_last_nm_provider','national_provider_id_provider_composite', 'urf_designation', 'provider_urf_designation']
subset_cols = ['urf_designation', 'provider_urf_designation']

provider_urf_data = df[subset_cols]

# Split into training and test sets
X_train_urf, X_test_urf = train_test_split(provider_urf_data, test_size=0.3, random_state=42)
print(X_train_urf.shape, X_test_urf.shape)



(7007, 2) (3003, 2)


In [31]:

# Perform One-Hot Encoding
encoder = OneHotEncoder(sparse_output=False)
X_trn_urf_encoded = encoder.fit_transform(X_train_urf)
print(f"Encoder feature names: {encoder.feature_names_in_}")
trn_encoded_df = pd.DataFrame(X_trn_urf_encoded, columns=encoder.get_feature_names_out(subset_cols))
X_tst_urf_encoded = encoder.transform(X_test_urf)


Encoder feature names: ['urf_designation' 'provider_urf_designation']


Train_Test_split

In [32]:
# Keep only numeric columns and drop missing values
df_numeric = df.select_dtypes(include='number').dropna()
print(df_numeric.shape)

# Split into training and test sets
X_train, X_test = train_test_split(df_numeric, test_size=0.3, random_state=42)
print(X_train.shape, X_test.shape)

(9997, 19)
(6997, 19) (3000, 19)


In [33]:
so_gaal = SO_GAAL(
    stop_epochs=20,     # 20 stop epochs means total = 60 epochs
    lr_d=0.01,          # learning rate for discriminator
    lr_g=0.0001,        # learning rate for generator
    momentum=0.9,       # momentum for SGD
    contamination=0.1   # 10% of data assumed to be anomalies
)


In [34]:
so_gaal.fit(X_train)


Epoch 1 of 60
Epoch 2 of 60
Epoch 3 of 60
Epoch 4 of 60
Epoch 5 of 60
Epoch 6 of 60
Epoch 7 of 60
Epoch 8 of 60
Epoch 9 of 60
Epoch 10 of 60
Epoch 11 of 60
Epoch 12 of 60
Epoch 13 of 60
Epoch 14 of 60
Epoch 15 of 60
Epoch 16 of 60
Epoch 17 of 60
Epoch 18 of 60
Epoch 19 of 60
Epoch 20 of 60
Epoch 21 of 60
Epoch 22 of 60
Epoch 23 of 60
Epoch 24 of 60
Epoch 25 of 60
Epoch 26 of 60
Epoch 27 of 60
Epoch 28 of 60
Epoch 29 of 60
Epoch 30 of 60
Epoch 31 of 60
Epoch 32 of 60
Epoch 33 of 60
Epoch 34 of 60
Epoch 35 of 60
Epoch 36 of 60
Epoch 37 of 60
Epoch 38 of 60
Epoch 39 of 60
Epoch 40 of 60
Epoch 41 of 60
Epoch 42 of 60
Epoch 43 of 60
Epoch 44 of 60
Epoch 45 of 60
Epoch 46 of 60
Epoch 47 of 60
Epoch 48 of 60
Epoch 49 of 60
Epoch 50 of 60
Epoch 51 of 60
Epoch 52 of 60
Epoch 53 of 60
Epoch 54 of 60
Epoch 55 of 60
Epoch 56 of 60
Epoch 57 of 60
Epoch 58 of 60
Epoch 59 of 60
Epoch 60 of 60


SO_GAAL(contamination=0.1, lr_d=0.01, lr_g=0.0001, momentum=0.9,
    stop_epochs=20)

In [35]:
# Get anomaly scores
train_scores = so_gaal.decision_function(X_train)
test_scores = so_gaal.decision_function(X_test)
print("SO-GAAL Train Scores Summary:\n", pd.Series(train_scores).describe())
print("\nSO-GAAL Test Scores Summary:\n", pd.Series(test_scores).describe())


SO-GAAL Train Scores Summary:
 count    6997.0
mean        0.0
std         0.0
min         0.0
25%         0.0
50%         0.0
75%         0.0
max         0.0
dtype: float64

SO-GAAL Test Scores Summary:
 count    3000.0
mean        0.0
std         0.0
min         0.0
25%         0.0
50%         0.0
75%         0.0
max         0.0
dtype: float64


In [1]:
%pip install tqdm

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
